# House Prices - Advanced Regression Techniques


<img src="https://storage.googleapis.com/kaggle-media/competitions/House%20Prices/kaggle_5407_media_housesbanner.png">

### Preços de casas - Técnicas Avançadas de Regressão

- Vamos utilizar o [dataset disponível no Kaggle](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview)
   

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Importando e visualizando base

In [2]:
treino = pd.read_csv('train.csv')
treino.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


In [3]:
teste = pd.read_csv('test.csv')
teste.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal


### Visualizando informações da base

In [4]:
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

## Analisando valores vazios

In [5]:
treino.isnull().sum().sort_values(ascending=False).head(20)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
Electrical         1
Id                 0
dtype: int64

### Visualizando a porcentagem de valores vazios em cada coluna

In [6]:
((treino.isnull().sum()/treino.shape[0]).sort_values(ascending=False)* 100) .head(20)

PoolQC          99.520548
MiscFeature     96.301370
Alley           93.767123
Fence           80.753425
MasVnrType      59.726027
FireplaceQu     47.260274
LotFrontage     17.739726
GarageYrBlt      5.547945
GarageCond       5.547945
GarageType       5.547945
GarageFinish     5.547945
GarageQual       5.547945
BsmtFinType2     2.602740
BsmtExposure     2.602740
BsmtQual         2.534247
BsmtCond         2.534247
BsmtFinType1     2.534247
MasVnrArea       0.547945
Electrical       0.068493
Id               0.000000
dtype: float64

### Apos uma analise, percebi que nas 4 primeiras colunas, "NA" é sim é uma informação, quer dizer ausencia.
- Podemos substituir os valores vazios por -1 ao inves de elimina-los

In [7]:
treino[['PoolQC','MiscFeature','Alley','Fence']] = treino[['PoolQC','MiscFeature','Alley','Fence']].fillna(-1)

### Vamos eliminar as colunas com mais de 10% de valores vazios para Treino e Teste

In [8]:
eliminar = treino.columns[treino.isnull().sum()/treino.shape[0] > 0.1]
eliminar

Index(['LotFrontage', 'MasVnrType', 'FireplaceQu'], dtype='object')

In [9]:
treino = treino.drop(eliminar,axis=1)
teste = teste.drop(eliminar,axis=1)

### Substituindo os valores vazios restantes

In [10]:
treino.isnull().sum().sort_values(ascending=False).head(3)

GarageFinish    81
GarageYrBlt     81
GarageQual      81
dtype: int64

In [11]:
treino = treino.fillna(-1)

#### Verificando novamente

In [12]:
treino.isnull().sum().sort_values(ascending=False).head(3)

Id              0
BedroomAbvGr    0
GarageYrBlt     0
dtype: int64

## Preparando para o modelo

### Selecionando apenas as colunas numéricas para o modelo

In [13]:
col_nr = treino.columns[treino.dtypes != 'object']
col_nr

Index(['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

### Criando nova base com esses valores


In [14]:
treino = treino.loc[:,col_nr]

## Criando nosso modelo

### Selecionando X e y

In [15]:
X = treino.drop('SalePrice',axis=1)
y = treino.SalePrice

### Treino e teste

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Regressão linear

In [18]:
from sklearn.linear_model import LinearRegression

In [19]:
reg_rl = LinearRegression().fit(X_train, y_train)

#### Previsão

In [20]:
y_pred_rl = reg_rl.predict(X_test)

## Arvore de regressão

In [21]:
from sklearn import tree

In [22]:
reg_ad = tree.DecisionTreeRegressor().fit(X_train, y_train)

#### Previsão

In [23]:
y_pred_ad = reg_ad.predict(X_test)

## Knn

In [24]:
from sklearn.neighbors import KNeighborsRegressor

In [25]:
reg_knn = KNeighborsRegressor(n_neighbors=2).fit(X_train, y_train)

#### Previsão

In [26]:
y_pred_knn = reg_knn.predict(X_test)

# Avaliando modelos

- Erro absoluto e erro quadrático

In [27]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

### Avaliando Erro da Regressão

In [28]:
print(mean_absolute_error(y_test, y_pred_rl))
print(mean_squared_error(y_test, y_pred_rl))

23763.18739306405
1533982883.4448233


### Avaliando Erro da árvore de decisão

In [29]:
print(mean_absolute_error(y_test, y_pred_ad))
print(mean_squared_error(y_test, y_pred_ad))

28129.682572614107
2575633264.30083


### Avaliando Erro do Knn

In [30]:
print(mean_absolute_error(y_test, y_pred_knn))
print(mean_squared_error(y_test, y_pred_knn))

33273.08298755187
2733937586.841286


## Fazendo a previsão para a base de teste da competição

In [31]:
# Mantendo apenas as colunas numéricas
col_nr_test = teste.columns[teste.dtypes != 'object']
teste = teste.loc[:,col_nr_test]

In [32]:
# Trocando todos os valores vazios por -1
teste = teste.fillna(-1)

In [33]:
# Verificando novamente
teste.isnull().sum().sort_values(ascending=False).head(3)

Id              0
MSSubClass      0
BedroomAbvGr    0
dtype: int64

## Agora podemos usar nosso modelo e ajustar os dados para usarmos no Kaggle

### Utilizando Regressao linear

In [34]:
y_pred = reg_rl.predict(teste)

### Podemos adicionar essa coluna de previsão na nossa base


In [35]:
teste['SalePrice'] = y_pred

### E extrair somente o Id e o SalePrice como pede o Kaggle


In [36]:
base_kaggle = teste[['Id','SalePrice']]
base_kaggle.head(3)

,Id,SalePrice
0,1461,122234.995960
1,1462,139178.263684
2,1463,169872.054251


### Podemos então exportar essa base


In [37]:
base_kaggle.to_csv('resultado1.csv',index=False)